<a href="https://colab.research.google.com/github/Bharatgaur/Projects/blob/main/AI%20Video%20Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install python-pptx gtts opencv-python pydub pillow moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 10.5 MB/s eta 0:00:00


In [6]:
!pip install pdf2image

In [10]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 1s (249 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...


In [11]:
# trying to fixing the error
# Error to fix ho gaya but ye kuch symbolls ko bol rahi thi hash underscore
import os
from pptx import Presentation
from gtts import gTTS
import moviepy.editor as mp
from pdf2image import convert_from_path

# === Step 1: Convert PPT to PDF and Extract Images ===
ppt_file = "/content/OOP_Introduction_Python.pptx"
pdf_file = "/content/OOP_Introduction_Python.pdf"
output_folder = "output"

os.makedirs(output_folder, exist_ok=True)

# Convert PPTX to PDF using LibreOffice (works in Colab)
!soffice --headless --convert-to pdf "{ppt_file}" --outdir "/content"

# Convert PDF pages to images (each slide as an image)
slides = convert_from_path(pdf_file)
image_files = []
for i, slide in enumerate(slides):
    image_path = f"{output_folder}/slide_{i+1}.png"
    slide.save(image_path, "PNG")
    image_files.append(image_path)

# === Step 2: Generate Audio for Each Slide ===
audio_files = []
for i, img_path in enumerate(image_files):
    slide_text = input(f"Enter narration text for Slide {i+1}: ")

    # Convert text to speech
    audio_path = f"{output_folder}/slide_{i+1}.mp3"
    tts = gTTS(text=slide_text, lang="en")
    tts.save(audio_path)
    audio_files.append(audio_path)

# === Step 3: Create Video ===
video_clips = []
for img_path, audio_path in zip(image_files, audio_files):
    image_clip = mp.ImageClip(img_path).set_duration(mp.AudioFileClip(audio_path).duration)
    audio_clip = mp.AudioFileClip(audio_path)
    video_clips.append(image_clip.set_audio(audio_clip))

# Combine all clips
final_video = mp.concatenate_videoclips(video_clips, method="compose")

# Save final video
video_output_path = "final_presentation.mp4"
final_video.write_videofile(video_output_path, codec="libx264", fps=24)

print(f"🎥 Video created successfully: {video_output_path}")


/bin/bash: line 1: soffice: command not found
Enter narration text for Slide 1: "Introduction to Object-Oriented Programming (OOP) in Python"   "Hello everyone! Welcome to this session on Object-Oriented Programming (OOP) in Python. Today, we will dive into the fundamental concepts of OOP, why it is important, and how we can use it effectively in Python. Whether you are a beginner or someone looking to refine your knowledge, this session will help you get a clear understanding of OOP. So, let’s begin!"
Enter narration text for Slide 2: What is OOP? "OOP is a programming paradigm based on objects, which contain data and methods."   "Object-Oriented Programming, or OOP, is a programming approach that organizes code around objects and classes instead of just functions and logic. Each object represents a real-world entity, holding data (attributes) and performing actions (methods). For example, think about a Car. A car has properties like color, brand, and model, and it can perform actions

MoviePy - Done.
Moviepy - Writing video final_presentation.mp4



Moviepy - Done !
Moviepy - video ready final_presentation.mp4
🎥 Video created successfully: final_presentation.mp4


In [13]:
import os
import re
import gtts
import moviepy.editor as mp
from pptx import Presentation
from pdf2image import convert_from_path
from pydub import AudioSegment
from pydub.playback import play

# === Step 1: Convert PPT to PDF and Extract Images ===
ppt_file = "/content/OOP_Introduction_Python.pptx"
pdf_file = "/content/OOP_Introduction_Python.pdf"
output_folder = "output"

os.makedirs(output_folder, exist_ok=True)

# Convert PPTX to PDF using LibreOffice
os.system(f'soffice --headless --convert-to pdf "{ppt_file}" --outdir "/content"')

# Convert PDF pages to images (each slide as an image)
slides = convert_from_path(pdf_file)
image_files = []
for i, slide in enumerate(slides):
    image_path = f"{output_folder}/slide_{i+1}.png"
    slide.save(image_path, "PNG")
    image_files.append(image_path)

# === Step 2: Generate Audio for Each Slide (Deep Male Voice) ===
def clean_text(text):
    """ Remove unwanted symbols from text. """
    text = re.sub(r'[_"#*<>@{}[\]()/\\]', '', text)  # Remove unwanted symbols
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

audio_files = []
for i, img_path in enumerate(image_files):
    slide_text = input(f"Enter narration text for Slide {i+1}: ")
    slide_text = clean_text(slide_text)

    # Convert text to speech (Deep Male Voice)
    audio_path = f"{output_folder}/slide_{i+1}.mp3"
    tts = gtts.gTTS(text=slide_text, lang="en", slow=False)
    tts.save(audio_path)

    # Lower the pitch (Make it sound more male-like)
    sound = AudioSegment.from_file(audio_path)
    deep_voice = sound._spawn(sound.raw_data, overrides={
        "frame_rate": int(sound.frame_rate * 0.8)  # Lower pitch
    }).set_frame_rate(sound.frame_rate)
    deep_voice.export(audio_path, format="mp3")

    audio_files.append(audio_path)

# === Step 3: Create Video ===
video_clips = []
for img_path, audio_path in zip(image_files, audio_files):
    image_clip = mp.ImageClip(img_path).set_duration(mp.AudioFileClip(audio_path).duration)
    audio_clip = mp.AudioFileClip(audio_path)
    video_clips.append(image_clip.set_audio(audio_clip))

# Combine all clips
final_video = mp.concatenate_videoclips(video_clips, method="compose")

# Save final video
video_output_path = "final_presentation.mp4"
final_video.write_videofile(video_output_path, codec="libx264", fps=24)

print(f"🎥 Video created successfully: {video_output_path}")

Enter narration text for Slide 1: "Hello everyone! Welcome to this session on Object-Oriented Programming (OOP) in Python. Today, we will dive into the fundamental concepts of OOP, why it is important, and how we can use it effectively in Python. Whether you are a beginner or someone looking to refine your knowledge, this session will help you get a clear understanding of OOP. So, let’s begin!"
Enter narration text for Slide 2: What is OOP? "OOP is a programming paradigm based on objects, which contain data and methods."   "Object-Oriented Programming, or OOP, is a programming approach that organizes code around objects and classes instead of just functions and logic. Each object represents a real-world entity, holding data (attributes) and performing actions (methods). For example, think about a Car. A car has properties like color, brand, and model, and it can perform actions like drive, brake, or accelerate. These properties and actions define an object. By using OOP, we make our pr

MoviePy - Done.
Moviepy - Writing video final_presentation.mp4



Moviepy - Done !
Moviepy - video ready final_presentation.mp4
🎥 Video created successfully: final_presentation.mp4


ValueError: max() arg is an empty sequence

In [17]:
!pip install edge-tts

  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=e9a49c6daccfbbb975efc63dbce842ac589f82155b411c88c9a267f9a87d717a
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt


In [19]:
!pip install moviepy pdf2image pydub gtts python-pptx

In [20]:
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [22]:
import edge_tts
voices = await edge_tts.list_voices()
for voice in voices:
    print(voice["ShortName"], "-", voice["Gender"])

af-ZA-AdriNeural - Female
af-ZA-WillemNeural - Male
sq-AL-AnilaNeural - Female
sq-AL-IlirNeural - Male
am-ET-AmehaNeural - Male
am-ET-MekdesNeural - Female
ar-DZ-AminaNeural - Female
ar-DZ-IsmaelNeural - Male
ar-BH-AliNeural - Male
ar-BH-LailaNeural - Female
ar-EG-SalmaNeural - Female
ar-EG-ShakirNeural - Male
ar-IQ-BasselNeural - Male
ar-IQ-RanaNeural - Female
ar-JO-SanaNeural - Female
ar-JO-TaimNeural - Male
ar-KW-FahedNeural - Male
ar-KW-NouraNeural - Female
ar-LB-LaylaNeural - Female
ar-LB-RamiNeural - Male
ar-LY-ImanNeural - Female
ar-LY-OmarNeural - Male
ar-MA-JamalNeural - Male
ar-MA-MounaNeural - Female
ar-OM-AbdullahNeural - Male
ar-OM-AyshaNeural - Female
ar-QA-AmalNeural - Female
ar-QA-MoazNeural - Male
ar-SA-HamedNeural - Male
ar-SA-ZariyahNeural - Female
ar-SY-AmanyNeural - Female
ar-SY-LaithNeural - Male
ar-TN-HediNeural - Male
ar-TN-ReemNeural - Female
ar-AE-FatimaNeural - Female
ar-AE-HamdanNeural - Male
ar-YE-MaryamNeural - Female
ar-YE-SalehNeural - Male
az-AZ-BabekNe

In [21]:
# Voice change ki hai or code ko fast kiya hai
import os
import re
import edge_tts
import asyncio
import moviepy.editor as mp
from pptx import Presentation
from pdf2image import convert_from_path

# === Step 1: Convert PPT to PDF and Extract Images ===
ppt_file = "/content/OOP_Introduction_Python.pptx"
pdf_file = "/content/OOP_Introduction_Python.pdf"
output_folder = "output"

os.makedirs(output_folder, exist_ok=True)

# Convert PPTX to PDF using LibreOffice (Linux)
os.system(f'soffice --headless --convert-to pdf "{ppt_file}" --outdir "/content"')

# Convert PDF pages to images (each slide as an image)
slides = convert_from_path(pdf_file, dpi=300)  # High-quality images
image_files = []
for i, slide in enumerate(slides):
    image_path = f"{output_folder}/slide_{i+1}.png"
    slide.save(image_path, "PNG")
    image_files.append(image_path)

# === Step 2: Generate Audio for Each Slide (Male Voice) ===
def clean_text(text):
    """ Remove unwanted symbols from text. """
    text = re.sub(r'[_"#*<>@{}[\]()/\\]', '', text)  # Remove unwanted symbols
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

async def generate_audio(text, output_file):
    """ Convert text to speech using Edge TTS (Microsoft AI). """
    voice = "en-GB-Wavenet-B"  # Male Voice
    tts = edge_tts.Communicate(text, voice, rate="+10%")  # Slightly faster
    await tts.save(output_file)

audio_files = []
for i, img_path in enumerate(image_files):
    slide_text = input(f"Enter narration text for Slide {i+1}: ")
    slide_text = clean_text(slide_text)

    audio_path = f"{output_folder}/slide_{i+1}.mp3"
    asyncio.run(generate_audio(slide_text, audio_path))

    audio_files.append(audio_path)

# === Step 3: Create Video (High-Quality, GPU Optimized) ===
video_clips = []
for img_path, audio_path in zip(image_files, audio_files):
    audio_clip = mp.AudioFileClip(audio_path)
    image_clip = mp.ImageClip(img_path).set_duration(audio_clip.duration)
    video_clips.append(image_clip.set_audio(audio_clip))

# Combine all clips
final_video = mp.concatenate_videoclips(video_clips, method="compose")

# Save final video (High Quality + GPU Acceleration)
video_output_path = "final_presentation.mp4"
final_video.write_videofile(video_output_path, codec="h264_nvenc", threads=8, preset="ultrafast", fps=30)

print(f"🎥 Video created successfully: {video_output_path}")


Enter narration text for Slide 1:  "Hello everyone! Welcome to this session on Object-Oriented Programming (OOP) in Python. Today, we will dive into the fundamental concepts of OOP, why it is important, and how we can use it effectively in Python. Whether you are a beginner or someone looking to refine your knowledge, this session will help you get a clear understanding of OOP. So, let’s begin!"


ValueError: Invalid voice 'en-GB-Wavenet-B'.